# Check landmarks

In [1]:
import pickle
import imageio
import azure
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import azure
%matplotlib inline

In [11]:
def load_landmarks(stim='id-1274_AU26-100_AU9-33', api='google'):

    stims = sorted(glob(f'../../../FEED_stimulus_frames/{stim}/*/texmap/frame*.png'))
    
    # number of landmarks: 27 (azure), 34 (google)
    n_lm = 27 if api == 'azure' else 34
    xy = np.zeros((30, n_lm, 2))  # 30 frames
    
    frames = [str(i).zfill(2) for i in range(1, 31)]
    for i, frame in enumerate(frames):
        info = stims[i].replace('.png', f'_api-{api}_annotations.pkl')
        with open(info, 'rb') as f_in:
            info = pickle.load(f_in)
        
        if api == 'azure':
            info = info[0].face_landmarks
            ii = 0
            for attr in dir(info):
                this_attr = getattr(info, attr)                
                if isinstance(this_attr, azure.cognitiveservices.vision.face.models._models_py3.Coordinate):
                    xy[i, ii, 0] = this_attr.x
                    xy[i, ii, 1] = this_attr.y
                    ii += 1

        elif api == 'google':
            info = info.face_annotations[0]
            for ii in range(len(info.landmarks)):
                xy[i, ii, 0] = info.landmarks[ii].position.x
                xy[i, ii, 1] = info.landmarks[ii].position.y
        else:
            raise ValueError("Choose api from 'google' and 'azure'.")
            
    return stims, xy

stims, xy = load_landmarks(api='azure')
xy

array([[[184.3, 321.6],
        [225.5, 316.5],
        [155.1, 307.2],
        ...,
        [303.1, 541.4],
        [303. , 535.2],
        [304.1, 515.7]],

       [[187.1, 322.6],
        [223.1, 315.5],
        [160.1, 306.6],
        ...,
        [290.7, 546. ],
        [294.9, 532.3],
        [297.5, 517.8]],

       [[191.3, 324.5],
        [230. , 316.9],
        [159.7, 305.1],
        ...,
        [298.6, 538.7],
        [297.8, 534.7],
        [299.8, 513.6]],

       ...,

       [[191.1, 320.8],
        [226.2, 313.6],
        [157.2, 306.9],
        ...,
        [298.9, 541.1],
        [299.2, 532.7],
        [300.8, 514.7]],

       [[192.9, 325.3],
        [229.9, 316.3],
        [161.5, 308.8],
        ...,
        [294.4, 541.5],
        [294.1, 529.2],
        [294.4, 513.9]],

       [[190.5, 321.9],
        [225.5, 313.2],
        [161.7, 306.9],
        ...,
        [302.1, 541.5],
        [300. , 531.7],
        [300.5, 515.2]]])

In [ ]:
def plot_face(imgs, landmarks, frame_nr=0):
    img = imageio.imread(imgs[frame_nr])
    plt.figure(figsize=(6, 8))
    plt.imshow(img)
    lm = landmarks[frame_nr, :, :]
    for i in range(lm.shape[0]):
        x, y = lm[i, :]
        plt.plot([x, x], [y, y], marker='o')
        
    plt.show();

In [ ]:
import ipywidgets
from ipywidgets import interact, fixed

slider = ipywidgets.IntSlider(min=0, max=29, step=1, value=0)
interact(plot_face, frame_nr=slider, imgs=fixed(stims), landmarks=fixed(xy));

In [ ]:
from scipy.ndimage import gaussian_filter
xy_std = (xy  - xy.mean(axis=0)) / xy.std(axis=0)

xy_filt = butter_bandpass_filter(data=xy_std[:, 0, :], lowcut=0.01, highcut=7, fs=30/1.25, order=5)
plt.plot(xy_filt)

In [ ]:
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

In [ ]:
from scipy.ndimage import gaussian_filter1d
gaussian_filter1d?